### WIP: Trying to implement the above in tf-slim
##### Implementation in Slim on MNIST
##### ... so far not working

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.metrics as metrics

##### MNIST data

In [ ]:
import numpy as np
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
Y_train = np_utils.to_categorical(Y_train, n_classes)
Y_test = np_utils.to_categorical(Y_test, n_classes)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#### Build the DenseNet model

In [ ]:
def bn_relu_conv(net, n_channels, drop, is_training=True):
     with slim.arg_scope([slim.conv2d], padding = 'SAME',
                        activation_fn=tf.nn.relu):
        net = tf.contrib.layers.batch_norm(net)
        net = slim.conv2d(net, n_channels, [3, 3])
        net = slim.dropout(net, drop, is_training=is_training)
        return net

def add_layer(net, n_channels, drop, is_training = True):
    net = bn_relu_conv(net, n_channels, drop, is_training)
    return net

def transition(net, n_channels, drop):
    net = bn_relu_conv(net, n_channels, drop)
    net = slim.max_pool2d(net, [2, 2], stride=[2, 2], padding='SAME')
    return net

def bn_relu_conv_model(input, start_channels, growth_rate, n_classes, levels, drop, is_training):
        
    with slim.arg_scope([slim.conv2d], padding = 'SAME', activation_fn=tf.nn.relu):
        with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.relu):
            n_channels = start_channels
            net = slim.conv2d(input, n_channels, [3, 3])

            for i in range(levels):
                add_layer(net, n_channels, drop, is_training)
                n_channels += growth_rate
            transition(net, n_channels, drop)

            net = tf.contrib.layers.batch_norm(net)
            net = slim.conv2d(net, n_channels, [3, 3])
            net = slim.flatten(net)
            net = slim.fully_connected(net, n_classes)
            return net

#### Training the model

In [ ]:
ckpt_dir = '/tmp/regression_model/'

with tf.Graph().as_default():
    
    start_channels = 16
    n_classes = 10
    drop = 0.2
    growth_rate = 12
    levels = 10
    
    ####have code for one hot labels, but the input data is already coded so this is unncessary in this instance
#one_hot_labels = slim.one_hot_encoding(Y_train, n_classes)

logits = bn_relu_conv_model(X_train, start_channels, growth_rate, n_classes, levels, drop, True)

loss = slim.losses.softmax_cross_entropy(logits, Y_train)

optimizer = tf.train.AdadeltaOptimizer(2e-5)

    train_op = slim.learning.create_train_op(loss, optimizer) 

    # Run the training inside a session.
    final_loss = slim.learning.train(
        train_op,
        logdir=ckpt_dir,
        number_of_steps=5000,
        save_summaries_secs=5,
        log_every_n_steps=500)
    
print("Finished training. Last batch loss:", final_loss)
print("Checkpoint saved in %s" % ckpt_dir)

#### Test Model Output

In [ ]:
with tf.Graph().as_default():
  
    # Create the model structure. (Parameters will be loaded below.)
    Y_out = bn_relu_conv_model(X_train, n_channels, growth_rate, 
                            n_classes, levels, drop, False)

    # Make a session which restores the old parameters from a checkpoint.
    sv = tf.train.Supervisor(logdir=ckpt_dir)
    with sv.managed_session() as sess:
        X_test, Y_out, Y_test = sess.run([X_test, Y_out, Y_test])

plt.scatter(X_test, Y_test, c='r');
plt.scatter(X_test, Y_out, c='b');
plt.title('red=true, blue=predicted')